In [6]:
import sys
import os
home=os.getcwd()
sys.path.append(home+'/Functions')
from ShowDF import *
import numpy as np
import pandas as pd
import sys

In [ ]:
BatchExtract_All_MS2_Spectra(DataFolder = 'Playground/Data',
                             saveFolder='ms2_spectra',
                             min_RT=0,
                             max_RT=1250,
                             min_mz=200,
                             max_mz=250,
                             minInt=1e4,
                             minPeaks=1,
                             LogFileName='LogFile_ms2.csv',
                             SFindicator='-ms2Summary.csv')

In [9]:
from ms2_SamplesAligment import *
ResultsFolder = 
ms2Folder = 
WorkingDirectory = 

AlignedSamplesDF = ms2_SamplesAligment(ProjectName = '',
                    All_SummMS2Table = All_SummMS2Table,
                    EdgesMat = EdgesMat,
                    SamplesNames = SamplesNames,
                    RT_tol = 30,
                    mz_Tol = 2e-3,
                    cos_tol = 0.9,
                    min_N_ms2_spectra = 3,
                    ToReplace = 'mzML-ms2Summary.xlsx',
                    ms2Folder = ms2Folder,
                    ToAdd = 'mzML',
                    Norm2One = True)